In [4]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 197 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 15.6 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests

def create_session(handle, password):
    """
    Logs in to Bluesky (bsky.social) with the given handle and password,
    and returns the 'accessJwt' from the createSession response.

    :param handle: Your Bluesky handle, e.g. "my-handle.bsky.social"
    :param password: Your Bluesky password
    :return: accessJwt (session token) as a string
    :raises: Exception if the login fails
    """

    # Replace this if you're using a different PDS (e.g., a custom server).
    endpoint = "https://bsky.social/xrpc/com.atproto.server.createSession"
    
    payload = {
        "identifier": handle,   # can be your handle, e.g. "my-handle.bsky.social"
        "password": password
    }

    # Make the POST request
    response = requests.post(endpoint, json=payload)

    # Check for a successful response
    if response.status_code == 200:
        data = response.json()
        # The field "accessJwt" is your session token
        return data["accessJwt"]
    else:
        # Raise an exception if something goes wrong (e.g., invalid credentials)
        raise Exception(f"Failed to create session: {response.status_code} {response.text}")



/Users/tobiasmichelsen/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
session_token = create_session("tobmic.bsky.social" , "qivyx7DDTyDkXba")

In [15]:
def get_user_profile(session_token, actor_handle):
    """
    Retrieves a user's profile given their handle (e.g., 'alice.bsky.social').
    Returns a JSON dict with profile info (DID, avatar, description, etc.).
    """
    endpoint = "https://bsky.social/xrpc/app.bsky.actor.getProfile"
    
    headers = {
        "Authorization": f"Bearer {session_token}"
    }
    
    params = {
        "actor": actor_handle
    }
    
    resp = requests.get(endpoint, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        raise Exception(f"Failed to get profile: {resp.status_code} {resp.text}")



In [16]:
get_user_profile(session_token, actor_handle='tobmic.bsky.social')

{'did': 'did:plc:2xyl3n4a6iwyypv4ykwna2vi',
 'handle': 'tobmic.bsky.social',
 'displayName': '',
 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:2xyl3n4a6iwyypv4ykwna2vi/bafkreifiqsw6lon2ysterag3ktcvq6tc2yr242vsbfwrmtccqxo2khwypq@jpeg',
 'associated': {'lists': 0,
  'feedgens': 0,
  'starterPacks': 0,
  'labeler': False},
 'viewer': {'muted': False, 'blockedBy': False},
 'labels': [],
 'createdAt': '2025-02-06T17:34:25.446Z',
 'indexedAt': '2025-02-06T17:34:25.446Z',
 'followersCount': 0,
 'followsCount': 1,
 'postsCount': 0}

In [20]:
def get_follows(session_token, actor_handle, limit=50, cursor=None):
    """
    Fetch the list of accounts that 'actor_handle' follows.
    Optional: specify 'limit' and 'cursor' for pagination.
    """
    endpoint = "https://bsky.social./xrpc/app.bsky.graph.getFollows"
    headers = {"Authorization": f"Bearer {session_token}"}
    
    params = {
        "actor": actor_handle,
        "limit": limit
    }
    if cursor:
        params["cursor"] = cursor
    
    resp = requests.get(endpoint, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()  # includes 'cursor' for next page
    else:
        raise Exception(f"Failed to get follows: {resp.status_code} {resp.text}")


def get_followers(session_token, actor_handle, limit=50, cursor=None):
    """
    Fetch the list of accounts following 'actor_handle'.
    """
    endpoint = "https://bsky.social/xrpc/app.bsky.graph.getFollowers"
    headers = {"Authorization": f"Bearer {session_token}"}
    
    params = {
        "actor": actor_handle,
        "limit": limit
    }
    if cursor:
        params["cursor"] = cursor
    
    resp = requests.get(endpoint, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        raise Exception(f"Failed to get followers: {resp.status_code} {resp.text}")



In [22]:
followers_data = get_followers(session_token, "biobab.bsky.social")
followers_data

{'followers': [],
 'subject': {'did': 'did:plc:kfqvsnaktgllblu77a7gb3x7',
  'handle': 'biobab.bsky.social',
  'displayName': '',
  'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:kfqvsnaktgllblu77a7gb3x7/bafkreicxok5ij2pswcqnjgzpkyftsvv3gxflrcyk2asysabtrvrrk7uw3u@jpeg',
  'viewer': {'muted': False, 'blockedBy': False},
  'labels': [],
  'createdAt': '2025-01-30T13:20:31.445Z',
  'indexedAt': '2025-01-30T13:20:31.445Z'}}

In [24]:
follows_data = get_follows(session_token, "biobab.bsky.social")
follows_data

{'follows': [{'did': 'did:plc:xlqcxpk53spbhlypj6wmvvke',
   'handle': 'popbase.tv',
   'displayName': 'Pop Base',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:xlqcxpk53spbhlypj6wmvvke/bafkreicieqzk3twxj6zeyd7gpm637zib277rkyciorvfu25whjhzn3542u@jpeg',
   'associated': {'chat': {'allowIncoming': 'all'}},
   'viewer': {'muted': False, 'blockedBy': False},
   'labels': [],
   'createdAt': '2024-09-03T22:04:10.910Z',
   'description': 'Pop Base is your best source for all pop culture related entertainment, news, award show coverage, chart updates, statistics and more. | email@popbase.tv',
   'indexedAt': '2025-02-07T09:28:50.044Z'},
  {'did': 'did:plc:z72i7hdynmk6r22z27h6tvur',
   'handle': 'bsky.app',
   'displayName': 'Bluesky',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:z72i7hdynmk6r22z27h6tvur/bafkreihagr2cmvl2jt4mgx3sppwe2it3fwolkrbtjrhcnwjk4jdijhsoze@jpeg',
   'associated': {'chat': {'allowIncoming': 'none'}},
   'viewer': {'muted': False,
    'blockedB

In [26]:
follows_data = get_follows(session_token, "sioldridge.bsky.social")
"Whats been done, fill the gap, innovative angle"
follows_data
rate limits

{'follows': [{'did': 'did:plc:m2ufb4ra3nh25vn32ybge6g3',
   'handle': 'mariesnyder.bsky.social',
   'displayName': 'Marie Snyder',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:m2ufb4ra3nh25vn32ybge6g3/bafkreicndqsx42crzpqynghwffwig3eh6p7dkodullcjrbuxsvdkuztmgy@jpeg',
   'associated': {'chat': {'allowIncoming': 'following'}},
   'viewer': {'muted': False, 'blockedBy': False},
   'labels': [],
   'createdAt': '2023-07-01T23:02:13.307Z',
   'description': 'Mask and relax with an N95! I post and repost about covid, climate, conflicts, cycling, psychology, philosophy, and politics. Longreads at https://apuffofabsurdity.blogspot.com/ and https://3quarksdaily.com/3quarksdaily/author/mariesnyder',
   'indexedAt': '2024-11-15T12:54:36.825Z'},
  {'did': 'did:plc:fqb2kfuwlskwq635xopujwza',
   'handle': 'jamesward81.bsky.social',
   'displayName': 'James Ward',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:fqb2kfuwlskwq635xopujwza/bafkreic4u5dhpdm6iosdrj3r5joyyhejer7y4